In [36]:
import json
import requests
from pprint import pprint
import difflib
from nltk.sentiment.vader import SentimentIntensityAnalyzer 

[nltk_data] Downloading package vader_lexicon to C:\Users\Sachin
[nltk_data]     Konan\AppData\Roaming\nltk_data...


True

In [4]:
with open('data.json') as json_data:
    d = json.load(json_data)
    data = d

In [5]:
len(data)

15

In [6]:
documents = {'documents' : []}
other_docs = {'documents': []}

for store in data:
    text = ''
    for i in store['reviews']:
        text += i[0]
        
    if(len(text) > 5120):
        done = False
        start = 0
        iteration = 0
        while(start < len(text)):
            end = start + 5120
            if(end >= len(text)):
                end = len(text)
            print(start, end)
            documents['documents'].append({'id': store['id'] +';' + str(iteration), 'language': 'en',
                                   'text': text[start: end]})
            iteration+=1
            start = end
    else:
        documents['documents'].append({'id': store['id'], 'language': 'en', 'text': text})
    
    other_docs['documents'].append({'id': store['id'], 'language': 'en', 'text': text})
    
    print('finished store')

0 5120
5120 10240
10240 13980
finished store
0 5120
5120 10240
10240 12342
finished store
0 5120
5120 10240
10240 10635
finished store
0 5120
5120 5270
finished store
finished store
finished store
0 5120
5120 7745
finished store
finished store
finished store
finished store
finished store
finished store
finished store
finished store
finished store


In [9]:
def getPopularPhones(business_id):
    pop_phones = []
    
    for store in key_phrases['documents']:
        if(business_id in store['id']):
            phones = {
                'samsung': [
                    's9',
                    's8',
                    's7',
                    'note',
                    's6', 
                ],
                'apple': ['iphone', 'ipad', 'ipod'],
                'motorola': ['motoX'],
                'google': ['Nexus'],
                'LG': ['g9', 'g5', 'g6', 'g3', 'g4'],
            }

            phrases = store['keyPhrases']
            frequencies = {}
            for word in phrases:
                for brand,v in phones.items():
                    for phone in v:
                        seq = difflib.SequenceMatcher(None,word.lower(),phone)
                        d = seq.ratio()*100
                        if(d > 0.75):
                            largest = None
                            smallest = None
                            if(len(phone) > len(word)):
                                largest = phone
                                smallest = word
                            else:
                                largest = word
                                smallest = phone

                            if(smallest.lower() in largest.lower()):
                                if(brand in frequencies.keys()):
                                    frequencies[phone] += 1
                                else:
                                    frequencies[phone] = 1
    return frequencies


text_analytics_base_url = "https://eastus.api.cognitive.microsoft.com/text/analytics/v2.0"
key_phrase_api_url = text_analytics_base_url + "/keyPhrases"

headers   = {'Ocp-Apim-Subscription-Key': '5c0a1002f18a454c9ebadfe86b72964f'}
response  = requests.post(key_phrase_api_url, headers=headers, json=documents)
key_phrases = response.json()

#print(key_phrases['documents'])
#print(getPopularPhones('N_v0tMRmqgTnnusrcVI3dw'))
for i in key_phrases['documents']:
    print(getPopularPhones(i['id']), i['id'])

{} wKKRoSg1pZQtQRYPoD7TXg;0
{} wKKRoSg1pZQtQRYPoD7TXg;1
{'iphone': 1} wKKRoSg1pZQtQRYPoD7TXg;2
{} 8hXzJzOb3K1eFor0oNrPWA;0
{} 8hXzJzOb3K1eFor0oNrPWA;1
{'iphone': 1, 'note': 1} 8hXzJzOb3K1eFor0oNrPWA;2
{'s7': 1, 'iphone': 1, 'note': 1} z-ldpf_9VxZjITfBb7Lnkg;0
{'Nexus': 1} z-ldpf_9VxZjITfBb7Lnkg;1
{} z-ldpf_9VxZjITfBb7Lnkg;2
{'s6': 1, 'iphone': 1} qiAckHMVZQScy2U9nvSCmQ;0
{} qiAckHMVZQScy2U9nvSCmQ;1
{'iphone': 1} 9oHGS1knSS3uAHsf2sqxRg
{'iphone': 1} oKrbc2dBvop1eo4dwdGk6Q
{'iphone': 1} GP2c4RUrfCPivCTVZ_VueA;0
{} GP2c4RUrfCPivCTVZ_VueA;1
{'s6': 1, 'iphone': 1} ZgkdwRzfT2QjQbzi34t96A
{'iphone': 1} N_v0tMRmqgTnnusrcVI3dw
{'iphone': 1, 'Nexus': 1} EGHoMbnG-vzwotSQz9oeqQ
{} 8XBfdAwp1Y_IrhFdSZaNzw
{'iphone': 1} Qh0Z1gyxugwZzAna4UsCKA
{} sSc4MPlUOPCEXEDQw_DISg
{'motoX': 1} jMCpYt9N6VqtjS-U2ubjZQ
{'note': 1} RlaaCtbydgyg8ItxIeuFRQ


In [72]:
from math import pi

import pandas as pd

from bokeh.io import output_notebook, show
from bokeh.palettes import Category20c, viridis
from bokeh.plotting import figure
from bokeh.transform import cumsum
from bokeh.palettes import Spectral6
from bokeh.models import ColumnDataSource


In [102]:
class GraphicsLib:
    def __init__(self, business):
        text_analytics_base_url = "https://eastus.api.cognitive.microsoft.com/text/analytics/v2.0"
        key_phrase_api_url = text_analytics_base_url + "/keyPhrases"

        headers   = {'Ocp-Apim-Subscription-Key': '5c0a1002f18a454c9ebadfe86b72964f'}
        response  = requests.post(key_phrase_api_url, headers=headers, json=documents)
        self.key_phrases = response.json()
        self.id = business
        self.sid = SentimentIntensityAnalyzer()
        
    def getPopularPhones(self):
        pop_phones = []
        
        business_id = self.id
        for store in self.key_phrases['documents']:
            if(business_id in store['id']):
                phones = {
                    'samsung': [
                        's9',
                        's8',
                        's7',
                        'note',
                        's6', 
                    ],
                    'apple': ['iphone', 'ipad', 'ipod'],
                    'motorola': ['motoX'],
                    'google': ['Nexus'],
                    'LG': ['g9', 'g5', 'g6', 'g3', 'g4'],
                }

                phrases = store['keyPhrases']
                frequencies = {}
                for word in phrases:
                    for brand,v in phones.items():
                        for phone in v:
                            seq = difflib.SequenceMatcher(None,word.lower(),phone)
                            d = seq.ratio()*100
                            if(d > 0.75):
                                largest = None
                                smallest = None
                                if(len(phone) > len(word)):
                                    largest = phone
                                    smallest = word
                                else:
                                    largest = word
                                    smallest = phone

                                if(smallest.lower() in largest.lower()):
                                    if(brand in frequencies.keys()):
                                        frequencies[phone] += 1
                                    else:
                                        frequencies[phone] = 1
        return frequencies
    
    def getServiceSentiment(self):
        business_id = self.id
        sentiment = {'pos': [0, []], 'neg': [0, []], 'neutral': [0, []]}
        for store in self.key_phrases['documents']:
            if(business_id in store['id']):
                phrases = store['keyPhrases']
                frequencies = {}
                phones = ['customer', 'service']
                for phone in phones:
                    for word in phrases:
                        seq = difflib.SequenceMatcher(None,word.lower(),phone)
                        d = seq.ratio()*100
                        if(d > 0.75):
                            largest = None
                            smallest = None
                            if(len(phone) > len(word)):
                                largest = phone
                                smallest = word
                            else:
                                largest = word
                                smallest = phone

                            if(smallest.lower() in largest.lower()):
                                polarity = self.sid.polarity_scores(largest)['compound']
                                if(polarity == 0.0):
                                    sentiment['neutral'][0] += 1
                                    sentiment['neutral'][1].append(largest)
                                elif(polarity < 0.0):
                                    sentiment['neg'][0] += 1
                                    sentiment['neg'][1].append(largest)
                                else:
                                    sentiment['pos'][0] += 1
                                    sentiment['pos'][1].append(largest)
                                    
        return sentiment
                    
    def getPhoneChart(self):
        x = self.getPopularPhones()
        
        data = pd.Series(x).reset_index(name='value').rename(columns={'index':'models'})
        data['angle'] = data['value']/data['value'].sum() * 2*pi
        data['color'] = viridis(len(x))

        p = figure(plot_height=350, title="Phone useage in Store %s" % (store_id), toolbar_location=None,
                   tools="hover", tooltips="@models: @value", x_range=(-0.5, 1.0))

        p.wedge(x=0, y=1, radius=0.4,
                start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
                line_color="white", fill_color='color', legend='models', source=data)

        p.axis.axis_label=None
        p.axis.visible=False
        p.grid.grid_line_color = None

        output_notebook()
        show(p)
        
    def getServiceSentimentChart(self):
        x = self.getServiceSentiment()

        source = ColumnDataSource(data=dict(sentiment= list(x.keys()), counts= [v[0] for k,v in x.items()],
                                            color=Spectral6, services = [';'.join(v[1]) for k,v in x.items()]))

        p = figure(x_range = list(x.keys()), title="Service Sentiment for %s" % (self.id),
                   toolbar_location=None, tools="hover", tooltips="@sentiment: @services")

        p.vbar(x='sentiment', top='counts', width=0.9, color='color', source=source)

        p.xgrid.grid_line_color = None
        
        output_notebook()
        show(p)
        
    def returnData(self):
        return self.key_phrases
    
    def getStoreIds(self):
        return list(map(lambda x: x['id'], self.returnData()['documents']))
                

In [104]:
a = GraphicsLib('z-ldpf_9VxZjITfBb7Lnkg')
a.getServiceSentimentChart()
print(a.getStoreIds())

Loading BokehJS ...

['wKKRoSg1pZQtQRYPoD7TXg;0', 'wKKRoSg1pZQtQRYPoD7TXg;1', 'wKKRoSg1pZQtQRYPoD7TXg;2', '8hXzJzOb3K1eFor0oNrPWA;0', '8hXzJzOb3K1eFor0oNrPWA;1', '8hXzJzOb3K1eFor0oNrPWA;2', 'z-ldpf_9VxZjITfBb7Lnkg;0', 'z-ldpf_9VxZjITfBb7Lnkg;1', 'z-ldpf_9VxZjITfBb7Lnkg;2', 'qiAckHMVZQScy2U9nvSCmQ;0', 'qiAckHMVZQScy2U9nvSCmQ;1', '9oHGS1knSS3uAHsf2sqxRg', 'oKrbc2dBvop1eo4dwdGk6Q', 'GP2c4RUrfCPivCTVZ_VueA;0', 'GP2c4RUrfCPivCTVZ_VueA;1', 'ZgkdwRzfT2QjQbzi34t96A', 'N_v0tMRmqgTnnusrcVI3dw', 'EGHoMbnG-vzwotSQz9oeqQ', '8XBfdAwp1Y_IrhFdSZaNzw', 'Qh0Z1gyxugwZzAna4UsCKA', 'sSc4MPlUOPCEXEDQw_DISg', 'jMCpYt9N6VqtjS-U2ubjZQ', 'RlaaCtbydgyg8ItxIeuFRQ']


In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize 
  
def tokenize(text):
    return word_tokenize(text)

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [ ]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

In [ ]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [ ]:
import random
text_data = []

for i in range(0, len(other_docs['documents'])):
    string = other_docs['documents'][i]['text']
    tokens = prepare_text_for_lda(string)
    if random.random() > .75:
        print(tokens)
        text_data.append(tokens)
        
print(text_data)

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [ ]:
import gensim
NUM_TOPICS = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=NUM_TOPICS)
for topic in topics:
    print(topic)